## Adding Data Using COCO Parser

In [ ]:
import os
import zipfile

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np

from luxonis_ml.data import LuxonisDataset, LuxonisLoader, LuxonisParser
from luxonis_ml.enums import DatasetType, LabelType

In [ ]:
# Delete dataset if exists

dataset_name = "coco_test"
if LuxonisDataset.exists(dataset_name):
    dataset = LuxonisDataset(dataset_name)
    dataset.delete_dataset()

### Download and extract data

In [ ]:
url = "https://drive.google.com/uc?id=1XlvFK7aRmt8op6-hHkWVKIJQeDtOwoRT"
output_zip = "../data/COCO_people_subset.zip"
output_folder = "../data/"

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
# Check if the data already exists
if not os.path.exists(output_zip) and not os.path.exists(
    os.path.join(output_folder, "COCO_people_subset")
):
    # Download the file
    gdown.download(url, output_zip, quiet=False)

    # Unzip the file
    with zipfile.ZipFile(output_zip, "r") as zip_ref:
        zip_ref.extractall(output_folder)
else:
    print("Data already exists. Exiting.")

### Convert from COCO using parsers

If any of your data is coming from one of the following formats, we have some automatic parsers to add your data to `LuxonisDataset`:
- COCO
- YOLO
- VOC
- Darknet
- CreateML
- and more!

In [ ]:
parser = LuxonisParser(dataset_name=dataset_name)
dataset = parser.parse_raw_dir(
    DatasetType.COCO,
    random_split=True,
    image_dir=os.path.join(output_folder, "person_val2017_subset"),
    annotation_path=os.path.join(output_folder, "person_keypoints_val2017.json"),
)

### Test Loader

In [ ]:
loader = LuxonisLoader(dataset, view="train")
for image, ann in loader:
    cls = ann[LabelType.CLASSIFICATION]
    box = ann[LabelType.BOUNDINGBOX]
    seg = ann[LabelType.SEGMENTATION]
    kps = ann[LabelType.KEYPOINT]

    h, w, _ = image.shape
    for b in box:
        cv2.rectangle(
            image,
            (int(b[1] * w), int(b[2] * h)),
            (int(b[1] * w + b[3] * w), int(b[2] * h + b[4] * h)),
            (255, 0, 0),
            2,
        )
    mask_viz = np.zeros((h, w, 3)).astype(np.uint8)
    for mask in seg:
        mask_viz[mask == 1, 2] = 255
    image = cv2.addWeighted(image, 0.5, mask_viz, 0.5, 0)

    for kp in kps:
        kp = kp[1:].reshape(-1, 3)
        for k in kp:
            cv2.circle(image, (int(k[0] * w), int(k[1] * h)), 2, (0, 255, 0), 2)

    plt.imshow(image)
    plt.axis("off")  # Optional: Hide axis
    plt.show()